In [35]:
import nltk
import sklearn

In [37]:
import numpy as np
import pandas as pd

In [38]:
Main_sentence = "A compiler is a special program that processes statements written in a high level language to machine level language"

In [39]:
news_headline1 = "A compiler is a special program that processes statements written in a high level language to machine level language"
news_headline2 = "A compiler is a program that processes high level language to machine level language"
news_headline3 = "A compiler is of high level language to machine level language"
news_headline4 = "Both apple and orange are fruit"

news_headlines = [news_headline1, news_headline2, news_headline3, news_headline4]

In [40]:
news_headline1_tokens = nltk.word_tokenize(news_headline1)
news_headline2_tokens = nltk.word_tokenize(news_headline2)
news_headline3_tokens = nltk.word_tokenize(news_headline3)
news_headline4_tokens = nltk.word_tokenize(news_headline4)
for words in [news_headline1_tokens, news_headline2_tokens, news_headline3_tokens, news_headline4_tokens]:
    print('tokens of the sentences: \n\n\n', words)

tokens of the sentences: 


 ['A', 'compiler', 'is', 'a', 'special', 'program', 'that', 'processes', 'statements', 'written', 'in', 'a', 'high', 'level', 'language', 'to', 'machine', 'level', 'language']
tokens of the sentences: 


 ['A', 'compiler', 'is', 'a', 'program', 'that', 'processes', 'high', 'level', 'language', 'to', 'machine', 'level', 'language']
tokens of the sentences: 


 ['A', 'compiler', 'is', 'of', 'high', 'level', 'language', 'to', 'machine', 'level', 'language']
tokens of the sentences: 


 ['Both', 'apple', 'and', 'orange', 'are', 'fruit']


In [41]:
from numpy import argmax

def transform(headlines):
    tokens = [w for s in headlines for w in s ]
    print()
    print('All Tokens:')
    print(tokens)

    results = []
    label_enc = sklearn.preprocessing.LabelEncoder()
    onehot_enc = sklearn.preprocessing.OneHotEncoder()
    
    encoded_all_tokens = label_enc.fit_transform(list(set(tokens)))
    encoded_all_tokens = encoded_all_tokens.reshape(len(encoded_all_tokens), 1)
    
    onehot_enc.fit(encoded_all_tokens)
    
    for headline_tokens in headlines:
        print()
        print('Original Input:', headline_tokens)
        
        encoded_words = label_enc.transform(headline_tokens)
        print('Encoded by Label Encoder:', encoded_words)
        
        encoded_words = onehot_enc.transform(encoded_words.reshape(len(encoded_words), 1))
        print('Encoded by OneHot Encoder:')
        print(encoded_words)

        results.append(np.sum(encoded_words.toarray(), axis=0))
    
    return results

transformed_results = transform([
    news_headline1_tokens, news_headline2_tokens, news_headline3_tokens, news_headline4_tokens])


All Tokens:
['A', 'compiler', 'is', 'a', 'special', 'program', 'that', 'processes', 'statements', 'written', 'in', 'a', 'high', 'level', 'language', 'to', 'machine', 'level', 'language', 'A', 'compiler', 'is', 'a', 'program', 'that', 'processes', 'high', 'level', 'language', 'to', 'machine', 'level', 'language', 'A', 'compiler', 'is', 'of', 'high', 'level', 'language', 'to', 'machine', 'level', 'language', 'Both', 'apple', 'and', 'orange', 'are', 'fruit']

Original Input: ['A', 'compiler', 'is', 'a', 'special', 'program', 'that', 'processes', 'statements', 'written', 'in', 'a', 'high', 'level', 'language', 'to', 'machine', 'level', 'language']
Encoded by Label Encoder: [ 0  6 10  2 18 17 20 16 19 22  9  2  8 12 11 21 13 12 11]
Encoded by OneHot Encoder:
  (0, 0)	1.0
  (1, 6)	1.0
  (2, 10)	1.0
  (3, 2)	1.0
  (4, 18)	1.0
  (5, 17)	1.0
  (6, 20)	1.0
  (7, 16)	1.0
  (8, 19)	1.0
  (9, 22)	1.0
  (10, 9)	1.0
  (11, 2)	1.0
  (12, 8)	1.0
  (13, 12)	1.0
  (14, 11)	1.0
  (15, 21)	1.0
  (16, 13)	

C:\Users\HELLO\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [42]:
print('Master Sentence: %s' % news_headlines[0])
for i, news_headline in enumerate(news_headlines):
    score = sklearn.metrics.pairwise.euclidean_distances([transformed_results[i]], [transformed_results[0]])[0][0]
    print('-----')
    print('Score: %.2f, Comparing Sentence: %s' % (score, news_headline))


Master Sentence: A compiler is a special program that processes statements written in a high level language to machine level language
-----
Score: 0.00, Comparing Sentence: A compiler is a special program that processes statements written in a high level language to machine level language
-----
Score: 2.24, Comparing Sentence: A compiler is a program that processes high level language to machine level language
-----
Score: 3.46, Comparing Sentence: A compiler is of high level language to machine level language
-----
Score: 5.57, Comparing Sentence: Both apple and orange are fruit


In [43]:
def calculate_position(values):
    x = []
    for pos, matrix in enumerate(values):
        if matrix > 0:
            x.append(pos)
    return x

"""
    Since scikit-learn can only compare same number of dimension of input. 
    Add padding to the shortest sentence.
"""
def padding(sentence1, sentence2):
    x1 = sentence1.copy()
    x2 = sentence2.copy()
    
    diff = len(x1) - len(x2)
    
    if diff > 0:
        for i in range(0, diff):
            x2.append(-1)
    elif diff < 0:
        for i in range(0, abs(diff)):
            x1.append(-1)
    
    return x1, x2    

y_actual = calculate_position(transformed_results[0])

print('Master Sentence: %s' % news_headlines[0])
for i, news_headline in enumerate(news_headlines):
    y_compare = calculate_position(transformed_results[i])
    x1, x2 = padding(y_actual, y_compare)
    score = sklearn.metrics.jaccard_similarity_score(x1, x2)
    print('-----')
    print('Score: %.2f, Comparing Sentence: %s' % (score, news_headline))

Master Sentence: A compiler is a special program that processes statements written in a high level language to machine level language
-----
Score: 1.00, Comparing Sentence: A compiler is a special program that processes statements written in a high level language to machine level language
-----
Score: 0.25, Comparing Sentence: A compiler is a program that processes high level language to machine level language
-----
Score: 0.06, Comparing Sentence: A compiler is of high level language to machine level language
-----
Score: 0.00, Comparing Sentence: Both apple and orange are fruit


In [44]:
from nltk import PorterStemmer
ps = PorterStemmer()

In [45]:
# importing modules 
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

ps = PorterStemmer() 
se = "A compiler is a special program that processes statements written in a high level language to machine level language"
# sentence = "Elon Musks Boring Co to build high speed airport link in Chicago"
wd = word_tokenize(se) 

for w in wd: 
	print(w, " : ", ps.stem(w)) 


A  :  A
compiler  :  compil
is  :  is
a  :  a
special  :  special
program  :  program
that  :  that
processes  :  process
statements  :  statement
written  :  written
in  :  in
a  :  a
high  :  high
level  :  level
language  :  languag
to  :  to
machine  :  machin
level  :  level
language  :  languag


In [46]:
from nltk import *

In [47]:
from nltk.stem import WordNetLemmatizer

In [48]:
word_list = nltk.word_tokenize(se)
lemmatizer = WordNetLemmatizer()
print(word_list,'\n','\n')
lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
print(lemmatized_output)

['A', 'compiler', 'is', 'a', 'special', 'program', 'that', 'processes', 'statements', 'written', 'in', 'a', 'high', 'level', 'language', 'to', 'machine', 'level', 'language'] 
 

A compiler is a special program that process statement written in a high level language to machine level language


In [49]:
print('Master Sentence: %s' % news_headlines[0])
for i, news_headline in enumerate(news_headlines):
    score = sklearn.metrics.pairwise.cosine_similarity([transformed_results[i]], [transformed_results[0]])[0][0]
    print('-----')
    print('Score: %.2f, Comparing Sentence: %s' % (score, news_headline))

Master Sentence: A compiler is a special program that processes statements written in a high level language to machine level language
-----
Score: 1.00, Comparing Sentence: A compiler is a special program that processes statements written in a high level language to machine level language
-----
Score: 0.90, Comparing Sentence: A compiler is a program that processes high level language to machine level language
-----
Score: 0.72, Comparing Sentence: A compiler is of high level language to machine level language
-----
Score: 0.00, Comparing Sentence: Both apple and orange are fruit
